# Import packages and API Setup

## Import packages

In [1]:
# Data manipulation
import numpy as np
import math
import pandas as pd
pd.set_option('display.max_columns', None) # Remove column display limit

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Import HTTP requests library
import requests


## API setup and configuration 

### Alpha Vantage

[Alpha Vantage API Documentation](https://www.alphavantage.co/documentation/#)

In [2]:
# Import dotenv_values and load the API key from .env file
from dotenv import dotenv_values 
secrets = dotenv_values("secrets/.env")
alpha_key = secrets["alphavantage_key"]

#### Search Endpoint

[Documentation](https://www.alphavantage.co/documentation/#symbolsearch)  

The Search Endpoint returns the best-matching symbols and market information based on keywords of your choice. The search results also contain match scores that provide you with the full flexibility to develop your own search and filtering logic.  

In [45]:
# Function to search for symbols based on a keyword
def av_search(keyword):
    # Base URL
    base_url = "https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={keyword}&apikey={apikey}"

    # API key for authentication
    api_key = alpha_key

    # Construct the full URL
    url = base_url.format(keyword=keyword, apikey=api_key)

    # Send HTTP GET request to fetch data
    response = requests.get(url)
    
    # Parse the response as JSON
    response_json = response.json()
    
    # Create a DataFrame from the 'bestMatches' section of the JSON response
    df_results = pd.DataFrame(response_json.get('bestMatches', []))

    return df_results

In [46]:
av_search("spy")

,1. symbol,2. name,3. type,4. region,5. marketOpen,6. marketClose,7. timezone,8. currency,9. matchScore
0,SPY,SPDR S&P 500 ETF Trust,ETF,United States,09:30,16:00,UTC-04,USD,1.0000
1,SPY1.FRK,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
2,SPY2.FRK,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
3,SPY4.FRK,SSgA SPDR S&P 400 US Mid Cap,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
4,SPY4.LON,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,United Kingdom,08:00,16:30,UTC+01,USD,0.6667
5,SPY1.DEX,SPDR® S&P 500 Low Volatility UCITS ETF,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
6,SPY2.DEX,SPDR® Dow Jones Global Real Estate UCITS ETF U...,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
7,SPY4.DEX,SSgA SPDR S&P 400 US Mid Cap,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
8,SPY4.PAR,SSgA SPDR S&P 400 US Mid Cap,ETF,Paris,09:00,17:30,UTC+02,EUR,0.6000
9,SPY5.DEX,SPDR® S&P 500 UCITS ETF,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000


#### Time Series Daily

[Documentation](https://www.alphavantage.co/documentation/#daily)  

This API returns raw (as-traded) daily time series (date, daily open, daily high, daily low, daily close, daily volume) of the global equity specified, covering 20+ years of historical data. The OHLCV data is sometimes called "candles" in finance literature. 

In [47]:
# Function to fetch daily stock data
def av_daily(symbol, output_size):
    # Base URL
    base_url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={output_size}&apikey={apikey}"

    # API key for authentication
    api_key = alpha_key

    # Construct the full URL
    url = base_url.format(symbol=symbol, output_size=output_size, apikey=api_key)

    # Send HTTP GET request to fetch data
    response = requests.get(url)

    # Parse the response as JSON
    data = response.json()

    # Extract the daily time series data, with a default to an empty dictionary
    time_series = data.get("Time Series (Daily)", {})

    # Create a DataFrame from the time series data
    ts_df = pd.DataFrame.from_dict(time_series, orient='index')

    # Convert the index to datetime for easier manipulation and analysis
    ts_df.index = pd.to_datetime(ts_df.index)

    # Return the resulting DataFrame
    return ts_df

In [48]:
av_daily("spy", "full")

,1. open,2. high,3. low,4. close,5. volume
2024-04-24,506.5600,507.3700,503.1300,505.4100,55928076
2024-04-23,501.7800,506.0900,499.5328,505.6500,64633620
2024-04-22,497.8300,502.3800,495.4300,499.7200,67961048
2024-04-19,499.4400,500.4550,493.8600,495.1600,102212587
2024-04-18,501.9800,504.1300,498.5600,499.5200,74548085
...,...,...,...,...,...
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900


#### Market News & Sentiment

[Documentation](https://www.alphavantage.co/documentation/#news-sentiment)

This API returns live and historical market news & sentiment data from a large & growing selection of premier news outlets around the world, covering stocks, cryptocurrencies, forex, and a wide range of topics such as fiscal policy, mergers & acquisitions, IPOs, etc.


The stock/crypto/forex symbols of your choice. For example: tickers=IBM will filter for articles that mention the IBM ticker; tickers=COIN,CRYPTO:BTC,FOREX:USD will filter for articles that simultaneously mention Coinbase (COIN), Bitcoin (CRYPTO:BTC), and US Dollar (FOREX:USD) in their content.

In [49]:
# Function to fetch news sentiment data
def av_news(tickers):
    # Base URL 
    base_url = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={tickers}&apikey={apikey}"

    # API key for authentication
    api_key = alpha_key

    # Construct the full URL
    url = base_url.format(tickers=tickers, apikey=api_key)

    # Send HTTP GET request to fetch data
    response = requests.get(url)

    # Parse the response as JSON
    data = response.json()

    # Normalize the 'feed' data into a DataFrame
    feed_df = pd.json_normalize(data['feed'])

    # Return the DataFrame with the news sentiment data
    return feed_df

In [50]:
av_news("S&P 500").head()

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,Forget Nvidia: 2 Artificial Intelligence ( AI...,https://www.fool.com/investing/2024/04/25/forg...,20240425T092900,[Anthony Di Pizio],Nvidia isn't the only fantastic opportunity in...,https://g.foolcdn.com/editorial/images/773990/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.224911,Somewhat-Bullish,"[{'ticker': 'NVDA', 'relevance_score': '0.1241..."
1,SentinelOne ( S ) Exceeds Market Returns: So...,https://www.zacks.com/stock/news/2261696/senti...,20240424T220019,[Zacks Equity Research],SentinelOne (S) closed at $21.57 in the latest...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.171072,Somewhat-Bullish,"[{'ticker': 'S', 'relevance_score': '0.526707'..."
2,SentinelOne Sees Relative Strength Rating Rise...,https://www.investors.com/ibd-data-stories/sen...,20240424T070000,"[INVESTOR'S BUSINESS DAILY, Investor's Busines...",The Relative Strength ( RS ) Rating for Sentin...,https://www.investors.com/wp-content/uploads/2...,Investors Business Daily,n/a,www.investors.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",0.303597,Somewhat-Bullish,"[{'ticker': 'CYBR', 'relevance_score': '0.3209..."
3,Endpoint Security Market size to record USD 12...,https://www.prnewswire.com/news-releases/endpo...,20240419T213000,[],Endpoint Security Market size to record USD 12...,None,PR Newswire,n/a,www.prnewswire.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.204600,Somewhat-Bullish,"[{'ticker': 'BB', 'relevance_score': '0.032513..."
4,High Wire Reports 2023 Revenue of $27.0 Millio...,https://www.globenewswire.com/news-release/202...,20240419T211000,[High Wire Networks Inc.],"Q1 2024 Revenue Outlook of $7.7 Million+, up 5...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.236476,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.0149..."


### Yahoo Finance